# Introduction to the cldrive API

In [1]:
import cldrive

## The OpenCL Environment

In [2]:
env = cldrive.make_env()
env

OpenCLEnvironment(ctx=<pyopencl.Context at 0x25a7c80 on <pyopencl.Device 'GeForce GTX 1080' on 'NVIDIA CUDA' at 0x24ae3c0>, <pyopencl.Device 'GeForce GTX 1080' on 'NVIDIA CUDA' at 0x24bda00>>, queue=<pyopencl.cffi_cl.CommandQueue object at 0x7f1a7996fc88>)

If `make_env()` can't find a suitable device, a `LookupError` is raised:

In [3]:
try:
    env = cldrive.make_env(platform_id=9999)
except LookupError:
    pass

In [4]:
for devtype in ['cpu', 'gpu', 'foobar']:
    try:
        cldrive.make_env(devtype=devtype)
        print(f'environment created for device type {devtype}')
    except LookupError:
        print(f'system has no {devtype}')
    except ValueError as e:
        print(e)

environment created for device type cpu
environment created for device type gpu
unsupported device type 'foobar'


To list the available platforms and devices, use `info()`:

In [5]:
cldrive.clinfo()

Host: Ubuntu 16.04 64bit
Platform 0: NVIDIA CUDA
    Device 0: GPU GeForce GTX 1080 375.39
    Device 1: GPU GeForce GTX 1080 375.39
Platform 1: Intel(R) OpenCL
    Device 0: CPU Intel(R) Xeon(R) CPU E5-2620 v4 @ 2.10GHz 1.2.0.25


## The Kernel Driver

In [6]:
kernel = """\
kernel void A(global int* data) {
    int tid = get_global_id(0);
    data[tid] *= 2.0;
}"""

cldrive.Driver(env, kernel)

kernel void A(global int* data) {
    int tid = get_global_id(0);
    data[tid] *= 2.0;
}

In [7]:
driver = cldrive.Driver(env, kernel, optimizations=False, debug=True)

OpenCL optimizations: off


In [8]:
import numpy as np

inputs = [np.arange(16)]
driver(inputs, gsize=(16,1,1), lsize=(8,1,1))

3-D global size 16 = [16, 1, 1]
3-D local size 8 = [8, 1, 1]


array([[ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30]], dtype=int32)

In [9]:
kernel = """\
kernel void A(global int* a, const int2 b) {
    const int tid = get_global_id(0);
    if (tid < get_global_size(0))
        a[tid] = a[tid] * 2.0 + b.x + b.y;
}"""

driver = cldrive.Driver(env, kernel)
driver([[0, 1, 2, 3], [10, 11]], gsize=(4, 1, 1), lsize=(1, 1, 1))

array([array([21, 23, 25, 27], dtype=int32), array([10, 11], dtype=int32)], dtype=object)